In [1]:
import pandas as pd
from konlpy.tag import Okt
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from math import log
from operator import itemgetter

In [2]:
dataset = pd.read_csv('D:/LikeLion/Code/Project2/Data/spell_check_label.csv',encoding='utf-8')

In [3]:
dataset

,reviewID,cleanliness,convenience,kindness,position,totalScore,content,date,hotelAddr,hotelGrade,hotelLoc,hotelName,label,sentence,fixed,probability
0,10737608,1,1,1,1,1,전철 다니는 소리때문에 잠을 못자겠습니다. 전철이 밤 늦게까지 새벽 일찍부터 다녀서...,2020-11-07,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,0,전철 다니는 소리때문에 잠을 못자겠습니다.,전철 다니는 소리 때문에 잠을 못 자겠습니다.,99.305508
1,10737608,1,1,1,1,1,전철 다니는 소리때문에 잠을 못자겠습니다. 전철이 밤 늦게까지 새벽 일찍부터 다녀서...,2020-11-07,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,0,전철이 밤 늦게까지 새벽 일찍부터 다녀서 몇시간 못잤습니다.,전철이 밤늦게까지 새벽 일찍부터 다녀서 몇 시간 못 잤습니다.,95.283567
2,10737608,1,1,1,1,1,전철 다니는 소리때문에 잠을 못자겠습니다. 전철이 밤 늦게까지 새벽 일찍부터 다녀서...,2020-11-07,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,0,그리고 어매니티가 핵구림.,그리고 어메니티가 핵구림.,95.697711
3,10736181,5,5,5,5,5,침대가 넓어서 너무 좋았어요! 접근성도 좋아서 추천합니다! 다만 화장실이 특이하게 ...,2020-11-07,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,침대가 넓어서 너무 좋았어요!,침대가 넓어서 너무 좋았어요!,99.883437
4,10736181,5,5,5,5,5,침대가 넓어서 너무 좋았어요! 접근성도 좋아서 추천합니다! 다만 화장실이 특이하게 ...,2020-11-07,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,접근성도 좋아서 추천합니다!,접근성도 좋아서 추천합니다!,99.991143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112058,2893006,5,5,5,5,5,깨끗하고 좋았어요,2018-08-12,서울특별시 서초구 서초동 1337-3 강남역 푸르지오시티,레지던스,강남역 5번출구 도보5분,어반플레이스 강남,1,깨끗하고 좋았어요,깨끗하고 좋았어요,99.989110
112059,2846155,4,4,4,4,4,만족합니다,2018-08-08,서울특별시 서초구 서초동 1337-3 강남역 푸르지오시티,레지던스,강남역 5번출구 도보5분,어반플레이스 강남,1,만족합니다,만족합니다,99.957941
112060,2845540,4,3,4,3,5,좋았습니다,2018-08-08,서울특별시 서초구 서초동 1337-3 강남역 푸르지오시티,레지던스,강남역 5번출구 도보5분,어반플레이스 강남,1,좋았습니다,좋았습니다,99.998638
112061,2840630,3,3,3,3,4,일방통행이라 주차하기별로,2018-08-07,서울특별시 서초구 서초동 1337-3 강남역 푸르지오시티,레지던스,강남역 5번출구 도보5분,어반플레이스 강남,0,일방통행이라 주차하기별로,일방통행이라 주차하기별로,97.179903


In [4]:
hotel_name = list(dataset.groupby('hotelName').count().index)
groups = dataset.groupby(dataset.hotelName)

In [5]:
len(hotel_name)

32

In [6]:
for i, name in enumerate(hotel_name):
    globals()['df_{}'.format(i)] = groups.get_group(name)

In [7]:
df_0_neg = df_0[df_0['label']==0].copy()

In [8]:
df_0_neg['fixed'] = df_0_neg['fixed'].apply(lambda x: re.sub(r"\d+"," ",x))

In [9]:
df_0_neg

,reviewID,cleanliness,convenience,kindness,position,totalScore,content,date,hotelAddr,hotelGrade,hotelLoc,hotelName,label,sentence,fixed,probability
86426,12735450,1,1,1,1,1,흐 음.,2021-05-02,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,흐 음.,흐 음.,65.692143
86429,12687833,4,4,4,4,4,만족합니다 주차장은 협소해요,2021-04-28,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,주차장은 협소해요,주차장은 협소해요,92.758669
86431,12671199,3,5,5,5,5,먼지가 좀 있지만 하루 있기에는 좋아요,2021-04-27,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,먼지가 좀 있지만 하루 있기에는 좋아요,먼지가 좀 있지만 하루 있기에는 좋아요,99.925895
86434,12644471,4,2,5,2,3,전화기가 없는게 가장 큰 불편함.뭘 물어보려는데 1층까지 가야했음.교통 위치접근성은...,2021-04-24,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,전화기가 없는게 가장 큰 불편함.뭘 물어보려는데 1층까지 가야했음.교통 위치접근성은...,전화기가 없는 게 가장 큰 불편함. 뭘 물어보려는데 층까지 가야 했음. 교통 위치...,62.882862
86435,12639619,3,2,1,1,1,서비스 형편 없어요 다시는 가고 싶지 않아요,2021-04-24,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,서비스 형편 없어요,서비스 형편없어요,95.918297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87897,7871842,5,5,4,4,4,체크아웃시에 데스크에 사람이 없어서.1분 기다림 그거 말고는 요즘 코로나로 관광객이...,2020-02-06,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,체크아웃시에 데스크에 사람이 없어서.,체크아웃 시에 데스크에 사람이 없어서.,97.993915
87899,7871842,5,5,4,4,4,체크아웃시에 데스크에 사람이 없어서.1분 기다림 그거 말고는 요즘 코로나로 관광객이...,2020-02-06,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,주차장은 조금 더 넓게 만드셨으면 주말에 늦게가면 꼭 기계식 이용함 주차장 안내 해...,주차장은 조금 더 넓게 만드셨으면 주말에 늦게 가면 꼭 기계식 이용함 주차장 안내해...,81.229856
87901,7857671,4,5,5,5,4,찾기 어렵지 않고 리버뷰 바로앞이 아파트와 폐골프연습장 이상해도 멀리보기엔 욕조에서...,2020-02-04,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,찾기 어렵지 않고 리버뷰 바로앞이 아파트와 폐골프연습장 이상해도 멀리보기엔 욕조에서...,찾기 어렵지 않고 리버뷰 바로 앞이 아파트와 폐골프연습장 이상해도 멀리 보기엔 욕조...,62.881339
87902,7857671,4,5,5,5,4,찾기 어렵지 않고 리버뷰 바로앞이 아파트와 폐골프연습장 이상해도 멀리보기엔 욕조에서...,2020-02-04,서울특별시 강서구 염창동 20-85,3성급,증미역 2번출구 도보 9분,JK 블라썸 호텔,0,더운물이 쉽게 나오지 않아요찬물나오기 한참후에 나옴.,더운물이 쉽게 나오지 않아요 찬물 나오기 한참 후에 나옴.,99.944473


In [50]:
document_count = len(df_0_neg)

In [10]:
okt = Okt()

In [11]:
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, norm=True, stem=True)
    return re.sub('\.','',' '.join(tokens_ko))

In [12]:
df_0_neg['fixed'] = df_0_neg['fixed'].apply(lambda x: okt_tokenizer(x))

In [13]:
documents = df_0_neg['fixed'].to_list()

In [14]:
def get_term_frequency(document, word_dict=None):
    if word_dict is None:
        word_dict = {}
    words = document.split()

    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

    return pd.Series(word_dict).sort_values(ascending=False)

In [25]:
def get_document_frequency(documents):
    dicts = []
    vocab = set([])
    df = {}

    for d in documents:
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())

    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
    
    return pd.Series(df, name='df').sort_values(ascending=False)

In [16]:
def get_tf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]

    stats = []

    for word, freq in vocab.items():
        tf_v = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_v += [tfs[idx][word]]
            
            else:
                tf_v += [0]

        stats.append((word, freq, *tf_v))

    column_name = ['word','totalFrequency']

    for i in range(1, len(docs)+1):
        column_name.append('document'+str(i))

    return pd.DataFrame(stats, columns=column_name).sort_values('totalFrequency', ascending=False)

In [17]:
# def get_tfidf(docs):
#     vocab = {}
#     tfs = []
#     for d in docs:
#         vocab = get_term_frequency(d, vocab)
#         tfs += [get_term_frequency(d)]
    
#     df = get_document_frequency(docs)

#     stats = []

#     for word, freq in vocab.items():
#         tfidfs = []
#         for idx in range(len(docs)):
#             if tfs[idx].get(word) is not None:
#                 tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            
#             else:
#                 tfidfs += [0]

#         stats.append((word, freq, *tfidfs, max(tfidfs)))
    
#     column_name = ['word','totalFrequency']

#     for i in range(1, len(docs)+1):
#         column_name.append('document'+str(i))
    
#     column_name.append('max')

#     return pd.DataFrame(stats, columns=column_name).sort_values('max', ascending=False)

In [26]:
tf_matrix = get_tf(documnets)
df_matrix = get_document_frequency(documnets)

ipykernel_launcher:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [19]:
df_matrix

가      116
이      112
하다     106
에       98
좋다      65
      ... 
체적       1
한다는      1
분들       1
양하다      1
장마       1
Length: 1235, dtype: int64

In [20]:
tf_matrix

,word,totalFrequency,document1,document2,document3,document4,document5,document6,document7,document8,...,document415,document416,document417,document418,document419,document420,document421,document422,document423,document424
0,하다,136,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,1,0,0,0
1,이,129,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,가,128,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,에,112,0,0,0,0,0,0,0,0,...,1,1,0,0,0,2,1,0,1,0
4,있다,74,0,0,2,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,연습장,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
812,이상하다,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
813,도르다,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
814,찬물,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
tf_matrix

,word,totalFrequency,document1,document2,document3,document4,document5,document6,document7,document8,...,document415,document416,document417,document418,document419,document420,document421,document422,document423,document424
0,하다,136,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,1,0,0,0
1,이,129,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,가,128,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,에,112,0,0,0,0,0,0,0,0,...,1,1,0,0,0,2,1,0,1,0
4,있다,74,0,0,2,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,연습장,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
812,이상하다,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
813,도르다,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
814,찬물,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [22]:
def get_ntf(matrix):
    max_btf = max(matrix['totalFrequency'])
    total_btf = sum(matrix['totalFrequency'])

    col_names = list(matrix.columns)[2:]

    matrix['ntf1'] = matrix['totalFrequency'].apply(lambda x: x/max_btf)

    matrix_ntf2 = matrix[col_names].copy()

    matrix_ntf2 = matrix_ntf2.apply(lambda x: x/total_btf , axis=1)
    
    matrix_ntf2['ntf2'] = matrix_ntf2.apply(sum, axis=1)

    matrix['ntf2'] = matrix_ntf2['ntf2']

    return matrix[['word','totalFrequency','ntf1','ntf2']]

In [39]:
ntf_matrix = get_ntf(tf_matrix)

In [53]:
def get_ntf_idf(ntf, df):

    ntf = ntf.set_index('word')
    ntf_idf = pd.concat([ntf, df], axis=1)


    def get_btfidf(scores):
        global document_count
        return (np.log(scores['totalFrequency']) + 1.0) * np.log(document_count/scores['df'])

    def get_ntf1idf(scores):
        global document_count
        return (np.log(scores['ntf1']) + 1.0) * np.log(document_count/scores['df'])

    def get_ntf2idf(scores):
        global document_count
        return (np.log(scores['ntf2']) + 1.0) * np.log(document_count/scores['df'])

    
    ntf_idf['btf_idf'] = ntf_idf.apply(get_btfidf, axis=1)
    ntf_idf['ntf1_idf'] = ntf_idf.apply(get_ntf1idf, axis=1)
    ntf_idf['ntf2_idf'] = ntf_idf.apply(get_ntf2idf, axis=1)

    btf_rank = ntf_idf['btf_idf'].sort_values(ascending=False)
    ntf1_rank = ntf_idf['ntf1_idf'].sort_values(ascending=False)
    ntf2_rank = ntf_idf['ntf2_idf'].sort_values(ascending=False)

    return btf_rank, ntf1_rank, ntf2_rank

In [54]:
btf_rank , ntf1_rank, ntf2_rank = get_ntf_idf(ntf_matrix, df_matrix)

In [61]:
btf_rank.to_csv('btf.csv', encoding='utf-8-sig')

In [62]:
ntf1_rank.to_csv('ntf1.csv',encoding='utf-8-sig')

In [63]:
ntf2_rank.to_csv('ntf2.csv', encoding='utf-8-sig')